In [1]:
from unsloth import FastLanguageModel

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name= "meta-llama/meta-Llama-3.1-8B-Instruct",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


c:\Users\hsavi\anaconda3\envs\stage_env\Lib\site-packages\unsloth_zoo\gradient_checkpointing.py:330: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"cuda:{i}") for i in range(n_gpus)])


==((====))==  Unsloth 2025.3.17: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    NVIDIA GeForce RTX 3070 Laptop GPU. Num GPUs = 1. Max memory: 8.0 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.3.17 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [2]:
import random
import json
import pandas as pd
from sampling.sampling_fallacies_detection import load_all_dataset, sample_data, format_prompt

In [3]:
paths = {
    'cocolofa': './Data_jsonl/cocolofa.jsonl',
    'mafalda': './Data_jsonl/mafalda.jsonl'
}
data = load_all_dataset(paths)
sampled_data, all_labels = sample_data(data)
cpt = 0
for k,v in sampled_data.items():
    for i in v:
        cpt += 1
print(f'lenght: {cpt}')
prt = format_prompt(sampled_data, all_labels)
df = pd.DataFrame().from_records(prt)
# for i in prt:
#     for p in i.get('prompt'):
#         print(p)
#     print('-----')
#     print(i.get('answer'))
#     print('=====')

lenght: 300


In [4]:
from datasets import Dataset
from unsloth.chat_templates import get_chat_template

data = Dataset.from_pandas(df)
# tok = get_chat_template(
#     tokenizer=tokenizer,
#     chat_template='llama',
# )
def formatting_prompt(ex):
    text = tokenizer.apply_chat_template(ex.get('prompt'),tokenize = False, add_generation_prompt = False)
    return { 'text': text, }

data1 = data.map(formatting_prompt, batched=True)
data1

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Dataset({
    features: ['datasets', 'prompt', 'answer', 'text'],
    num_rows: 300
})

In [5]:
from transformers import TextStreamer
# [0].outputs[0].text
FastLanguageModel.for_inference(model)
text_streamer = TextStreamer(tokenizer)

def gen(msg, model):
    txt = tokenizer.apply_chat_template(
        msg,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to('cuda')
    output = model.generate(
        txt,
        streamer=text_streamer,
        max_new_tokens=128,
        pad_token_id=tokenizer.eos_token_id
    )
    return output

msg_test = prt[0].get('prompt')
ground = prt[0].get('answer')
output = gen(msg_test, model)

print(tokenizer.batch_decode(output))
print(ground)
# txt = tokenizer.apply_chat_template(
#     msg_test,
#     add_generation_prompt=True,
#     return_tensors="pt"
# ).to('cuda')
# out = model.generate(
#     txt,
#     streamer=text_streamer,
#     max_new_tokens=128,
#     pad_token_id=tokenizer.eos_token_id
# )


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are an expert in argumentation. Your task is to determine the type of fallacy in the given Sentence. The fallacy would be in the Fallacy Set. Utilize the Title and the Full Text as context to support your decision and provide an explanation of the reasoning behind your determination.<|eot_id|><|start_header_id|>user<|end_header_id|>

Fallacy: {'straw man', 'appeal to ridicule','slippery slope', 'appeal to majority', 'guilt by association', 'none', 'equivocation', 'false causality', 'appeal to tradition', 'hasty generalization', 'false analogy', 'ad populum', 'causal oversimplification', 'circular reasoning', 'appeal to authority', 'ad hominem', 'appeal to worse problems', 'appeal to fear', 'false dilemma', 'appeal to nature'}
Title: Malaysian lawyers behind ‘walk for judicial independence’ facing police probe
Sentence:  Once investigations happen like this for

In [6]:
#You are an expert in argumentation. Your task is to determine the relation from #Argument 1 to Argument 2 as either "attack" or "support". That is, does #Argument 1 attack or support Argument 2 ?
#
#Think about the reasoning process first and then give an answer.
#Respond in the following format with your thought process in the reasoning #section and only attack or support in the answer section:
#<reasoning>
#...
#</reasoning>
#<answer>
#...
#</answer>
#"""
#
#def format_user_prompt(arg1, arg2):
#    prompt = ""
#    prompt += "Argument 1 : "+arg1+"\n"
#    prompt += "Argument 2 : "+arg2+"\n"
#    return prompt
#
#def get_dataset_prompts(split = "train") -> Dataset:
#    if split == "train":
#        data = load_dataset("csv", data_files='kialoPairs.csv', split='train#[:80%]') # type: ignore
#    elif split == "test":
#        data = load_dataset("csv", data_files='kialoPairs.csv', split='train#[91%:92%]') # type: ignore
#    elif split == "val":
#        # Load full dataset
#        data = load_dataset("csv", data_files='kialoPairs.csv', split='train')
#        # Select 100 rows starting at around 90'th % index
#        start_idx = int(0.90 * len(data))
#        offset = -14
#        data = data.select(range(start_idx+offset, min(start_idx + 100, len#(data))+offset))
#    data = data.map(lambda x: { # type: ignore
#        'prompt': [
#            {'role': 'system', 'content': SYSTEM_PROMPT},
#            {'role': 'user', 'content': format_user_prompt(x["argSrc"], x#["argTrg"])}
#        ],
#        'answer': x['relation']
#    }) # type: ignore
#    return data # type: ignore
#
#from vllm import SamplingParams
#MAX_COMPLETION_LENGTH = 512
#
#SAMPLING_PARAMS = SamplingParams(
#    temperature = 0.8,
#    top_p = 0.95,
#    max_tokens = MAX_COMPLETION_LENGTH,
#)
#
#def query_llm_classification(arg1:str, arg2:str, sampling_params = #SAMPLING_PARAMS):
#    """
#    Queries the language model with the task specific prompt (a pair of #arguments) and returns the extracted classification answer
#    """
#    text = tokenizer.apply_chat_template([
#        {"role" : "system", "content" : SYSTEM_PROMPT},
#        {"role" : "user", "content" : format_user_prompt(arg1, arg2)},
#    ], tokenize = False, add_generation_prompt = True)
#
#    output = model.fast_generate(
#        text,
#        sampling_params = sampling_params,
#        lora_request = model.load_lora("grpo_saved_lora"),
#    )[0].outputs[0].text
#
#    # The output is valid (i.e. parsable) if both sets of tags are present in #the response
#    valid_extract = (output.count('<answer>') + output.count('</answer>') == 2) #and (output.count('<reasoning>') + output.count('</reasoning>') == 2)
#
#    return valid_extract, extract_xml_answer(output), extract_xml_reasoning(output)